In [1]:
# install.packages(c("RDFTensor", "dplyr"))

library(dplyr)
library(RDFTensor)

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'RDFTensor' was built under R version 3.6.3"Loading required package: Matrix
Loading required package: pracma
Warning message:
"package 'pracma' was built under R version 3.6.3"
Attaching package: 'pracma'

The following objects are masked from 'package:Matrix':

    expm, lu, tril, triu

Loading required package: doParallel
Warning message:
"package 'doParallel' was built under R version 3.6.3"Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


In [2]:
setwd("D://LA//ATI Data")

In [3]:
Auth <- read.table(file="Summaries//Just_Auth//LA.txt", header=T, sep=",")
Auth <- Auth[, -1]

In [4]:
head(Auth)
dim(Auth)
Auth %>% mutate(across(where(is.factor), as.character)) -> Auth

str(Auth)

UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
User035855,Comp808475,Comp081330,TGS,0,17
Comp655251$,Comp655251,ActiveDirectory,NetworkLogon,0,350
User762066,Comp306129,ActiveDirectory,TGS,0,22
User384215,Comp095190,EnterpriseAppServer,NetworkLogon,0,35
User043263,Comp883307,Comp384394,TGS,0,2
User631552,Comp621781,Comp915658,NetworkLogon,0,1


[1] 15953681        6

'data.frame':	15953681 obs. of  6 variables:
 $ UserName  : chr  "User035855" "Comp655251$" "User762066" "User384215" ...
 $ SrcDevice : chr  "Comp808475" "Comp655251" "Comp306129" "Comp095190" ...
 $ DstDevice : chr  "Comp081330" "ActiveDirectory" "ActiveDirectory" "EnterpriseAppServer" ...
 $ AuthType  : chr  "TGS" "NetworkLogon" "TGS" "NetworkLogon" ...
 $ Failure   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ DailyCount: int  17 350 22 35 2 1 14 78 2 4 ...


### Trying out something different

In [79]:
train <- sample(1:nrow(Auth), size=nrow(Auth)*10^-4, replace=FALSE)
U17<- Auth[train, ]

head(U17)
dim(U17)

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
4519773,User667374,Comp920140,ActiveDirectory,TGT,0,2
464434,Comp237460$,Comp237460,ActiveDirectory,NetworkLogon,0,349
10664363,User736129,Comp107130,Comp916004,NetworkLogon,0,10
14580771,Comp371416$,Comp371416,ActiveDirectory,NetworkLogon,0,335
1316767,User461881,Comp981062,Comp981062,TGS,0,8
15612147,User680793,Comp738540,ActiveDirectory,NetworkLogon,0,215


[1] 1595    6

In [80]:
t1<- tapply(U17[,1], U17[,1])
t2<- tapply(U17[,2], U17[,2])
t3<- tapply(U17[,3], U17[,3])
t4<- tapply(U17[,4], U17[,4])
t5<- tapply(U17[,5], U17[,5])
t6<- tapply(U17[,6], U17[,6])

T<- as.data.frame(cbind(t1,t2,t3,t4,t5,t6))
head(T)

t1,t2,t3,t4,t5,t6
1134,1236,1,9,1,2
124,320,1,4,1,176
1201,156,240,4,1,10
200,507,1,4,1,171
951,1326,253,8,1,8
1147,981,1,4,1,126


In [86]:
T<- as.matrix(T)
X=list(subs=T, vals=rep(1, nrow(T)), size=c(length(unique(t1)), length(unique(t2)), length(unique(t3)), length(unique(t4)), length(unique(t5)), length(unique(t6))))

print(c(length(unique(t1)), length(unique(t2)), length(unique(t3)), length(unique(t4)), length(unique(t5)), length(unique(t6))))

P4<- cp_apr(X, 2, opts=list(alg='pdnr'))
Pt4 <- as.data.frame(P4$M$u[1])
pt4<- Pt4[order(Pt4[,1], decreasing=TRUE), ]

[1] 1462 1355  260   11    2  230
[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 6154, KKT viol = 1.00e+00, obj = -4.79137500e+05, nz: 2127, time:2.94"
[1] "   2. Ttl Inner Its: 20738, KKT viol = 7.27e+03, obj = -1.65571339e+04, nz: 2969, time:10.19"
[1] "   3. Ttl Inner Its: 3077, KKT viol = 2.19e+00, obj = -1.65571339e+04, nz: 2969, time:10.52"
[1] "   4. Ttl Inner Its: 4788, KKT viol = 2.19e+00, obj = -1.64203685e+04, nz: 2997, time:11.52"
[1] "   5. Ttl Inner Its: 3123, KKT viol = 1.93e-01, obj = -1.64201184e+04, nz: 2998, time:11.86"
[1] "   6. Ttl Inner Its: 5992, KKT viol = 1.03e+00, obj = -1.64177610e+04, nz: 3010, time:13.27"
[1] "   7. Ttl Inner Its: 3088, KKT viol = 1.49e+00, obj = -1.64103627e+04, nz: 3012, time:13.61"
[1] "   8. Ttl Inner Its: 3077, KKT viol = 1.01e-02, obj = -1.64103627e+04, nz: 3012, time:13.94"
[1] "   9. Ttl Inner Its: 7287, KKT viol = 2.03e-02, obj

In [99]:
head(pt4)
dim(pt4[pt4$X1>0.00128, ])
R<- as.numeric(rownames(pt4[pt4$X1>0.00128, ]))
R

,X1,X2
2,0.008894499,0.044554922
1,0.005082823,0.001237594
1141,0.002541419,0.000000000
1186,0.002541284,0.000000000
1015,0.002541281,0.000000000
1400,0.002541281,0.000000000


[1] 33  2

[1]    2    1 1141 1186 1015 1400 1219 1019 1336 1105 1111  672 1435  940  980
[16]  960   43 1006 1177 1266 1420  891  877  314 1052  983  543  435  884  625
[31]  606  798  628

In [102]:
T<- as.data.frame(T)
data<- cbind(U17, T)
data<- data[data$t1%in%R, ]

head(data)
dim(data)
L<- unique(data$UserName)

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount,t1,t2,t3,t4,t5,t6
7717125,User969583,Comp636287,Comp797582,NetworkLogon,0,1,1435,834,214,4,1,1
5637233,User496897,Comp388275,Comp081330,NetworkLogon,0,27,983,525,27,4,1,27
15879011,Administrator,Comp749049,Comp788417,TGS,0,2,1,994,210,8,1,2
8927698,Scanner,Comp296454,Comp467728,NetworkLogon,0,63,543,399,129,4,1,55
11588351,Anonymous,Comp844043,Comp747943,NetworkLogon,0,18,2,1135,196,4,1,18
1574466,User495233,Comp073023,Comp805594,NetworkLogon,0,6,980,108,216,4,1,6


[1] 126  12

In [103]:
setwd("D://LA//ATI Data")
AU <- read.table(file="AuthUserNames.txt", header=F)
AU <- as.vector(AU$V1)
AU <- unique(AU)

Y<- L[L%in%AU==TRUE]

print(paste(length(Y)/length(L)*100, "% of the list of usernames detected features in the AU list."))

[1] "6.06060606060606 % of the list of usernames detected features in the AU list."


### User179746

In [74]:
U17 <- Auth[Auth$UserName=='User179746', ]

In [6]:
df<- as.data.frame(cbind(Src=U17$SrcDevice, Dst=U17$DstDevice, Fail=U17$Failure))

In [7]:
head(df)
dim(df)

Src,Dst,Fail
Comp599155,None,0
Comp599155,None,0
Comp599155,None,0
Comp679250,Comp599155,0
Comp547871,ActiveDirectory,1
Comp679250,Comp599155,0


[1] 57  3

In [8]:
SrcT <- tapply(df$Src, df$Src)
DstT <- tapply(df$Dst, df$Dst)

dm <- as.matrix(cbind(SrcT, DstT, df$Fail))

In [9]:
head(dm)

head(df)

SrcT,DstT,
3,25,1
3,25,1
3,25,1
4,14,1
2,1,2
4,14,1


Src,Dst,Fail
Comp599155,None,0
Comp599155,None,0
Comp599155,None,0
Comp679250,Comp599155,0
Comp547871,ActiveDirectory,1
Comp679250,Comp599155,0


In [72]:
subs=matrix(c(5,1,1,
              3,1,2,
              1,1,3,
              2,1,3,
              4,1,3,
              6,1,3,
              1,1,4,
              2,1,4,
              4,1,4,
              6,1,4,
              1,2,1,
              3,2,1,
              5,2,1),byrow=TRUE,ncol=3)
X=list(subs=subs,vals=rep(1,nrow(subs)),size=c(6,2,4))

P1=cp_apr(X,2,opts=list(alg='mu'))
print(P1$M)

[1] "CP_APR: mu"
[1] " Iter    1: Inner Its = 30 KKT violation = 9.206783e-02, nViolations =  0"
[1] " Iter    2: Inner Its = 19 KKT violation = 5.651181e-05, nViolations =  0"
[1] " Iter    3: Inner Its =  5 KKT violation = 5.651143e-05, nViolations =  0"
[1] " Iter    4: Inner Its =  3 KKT violation = 5.651143e-05, nViolations =  0"
[1] "Exiting because all subproblems reached KKT tol."
[1] "==========================================="
[1] " Final log-likelihood = -1.700256e+01 "
[1] " Final least squares fit = 5.856965e-01 "
[1] " Final KKT violation = 5.6511434e-05"
[1] " Total inner iterations = 57"
[1] " Total execution time = 1.31 secs"
$lambda
[1] 4.000034 8.999966

$u
$u[[1]]
             [,1]     [,2]
[1,] 2.499981e-01 0.222223
[2,] 4.168178e-15 0.222223
[3,] 2.500059e-01 0.111108
[4,] 1.329797e-14 0.222223
[5,] 4.999960e-01 0.000000
[6,] 1.721164e-14 0.222223

$u[[2]]
          [,1] [,2]
[1,] 0.2500063    1
[2,] 0.7499937    0

$u[[3]]
             [,1]      [,2]
[1,] 9.9998

In [50]:
M <- list(subs=dm, vals=rep(1, nrow(dm)), size=c(length(unique(SrcT)), length(unique(DstT)), 2))
invisible(M)

tfm<- getTensor(trp=dm)

In [56]:
set.seed(5)
P1 <- cp_apr(M,2,opts=list(alg='mu'))

print(P1$M)

[1] "CP_APR: mu"
[1] " Iter    1: Inner Its = 23 KKT violation = 1.335418e-01, nViolations =  0"
[1] " Iter    2: Inner Its =  8 KKT violation = 6.241433e-07, nViolations =  0"
[1] " Iter    3: Inner Its =  3 KKT violation = 6.241433e-07, nViolations =  0"
[1] "Exiting because all subproblems reached KKT tol."


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = -2.925011e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 6.2414330e-07"
[1] " Total inner iterations = 34"
[1] " Total execution time = 0.21 secs"
$lambda
[1] 48  9

$u
$u[[1]]
              [,1]         [,2]
[1,] 2.600441e-128 1.111111e-01
[2,]  6.250000e-02 3.213536e-09
[3,]  9.375000e-01 6.934926e-08
[4,] 9.044691e-118 8.888888e-01

$u[[2]]
               [,1]         [,2]
 [1,]  4.166667e-02 3.470756e-34
 [2,]  4.166667e-02 2.555932e-14
 [3,]  2.083333e-02 1.272722e-14
 [4,]  2.083333e-02 6.725709e-15
 [5,]  4.166667e-02 3.273877e-14
 [6,]  2.083333e-02 1.162181e-14
 [7,]  2.083333e-02 6.695594e-15
 [8,]  4.166667e-02 1.690307e-14
 [9,]  2.083333e-02 1.952144e-14
[10,]  2.083333e-02 1.602380e-14
[11,]  2.083333e-02 3.523004e-14
[12,]  4.166667e-02 4.726771e-15
[13,] 3.393511e-156 2.222222e-01
[14,] 9.979302e-156 7.777778e-01
[15,]  4.166667e-02 1.621977e-14
[16,]  1.250000e-01 4.

In [67]:
P1_2<- cp_apr(M,2,opts=list(alg='pdnr'))

print(P1_2$M)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 62, KKT viol = 2.76e+00, obj = -5.71527178e+01, nz: 7, time:0.58"
[1] "   2. Ttl Inner Its: 134, KKT viol = 2.63e+00, obj = -3.39026314e+01, nz: 11, time:0.75"
[1] "   3. Ttl Inner Its: 42, KKT viol = 9.96e-01, obj = -2.95267136e+01, nz: 12, time:0.76"
[1] "   4. Ttl Inner Its: 53, KKT viol = 3.13e-02, obj = -2.92786951e+01, nz: 26, time:0.80"
[1] "   5. Ttl Inner Its: 62, KKT viol = 9.74e-03, obj = -2.92501049e+01, nz: 29, time:0.83"
[1] "   6. Ttl Inner Its: 31, KKT viol = 6.77e-05, obj = -2.92501049e+01, nz: 29, time:0.84"


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = -2.925010e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 6.7709872e-05"
[1] " Total inner iterations = 384"
[1] " Total execution time = 0.84 secs"
$lambda
[1]  8.999972 47.999815

$u
$u[[1]]
          [,1]       [,2]
[1,] 0.1111111 0.00000000
[2,] 0.0000000 0.06249999
[3,] 0.0000000 0.93750001
[4,] 0.8888889 0.00000000

$u[[2]]
           [,1]       [,2]
 [1,] 0.0000000 0.04166741
 [2,] 0.0000000 0.04166740
 [3,] 0.0000000 0.02083371
 [4,] 0.0000000 0.02083370
 [5,] 0.0000000 0.04166401
 [6,] 0.0000000 0.02083370
 [7,] 0.0000000 0.02083370
 [8,] 0.0000000 0.04166740
 [9,] 0.0000000 0.02083370
[10,] 0.0000000 0.02083351
[11,] 0.0000000 0.02083371
[12,] 0.0000000 0.04166740
[13,] 0.2222223 0.00000000
[14,] 0.7777777 0.00000000
[15,] 0.0000000 0.04166676
[16,] 0.0000000 0.12499877
[17,] 0.0000000 0.02083347
[18,] 0.0000000 0.04166741
[19,] 0.0000000 0.04166740
[20,] 0.0000000 0.0416674

In [92]:
P1_3<- cp_apr(M,2,opts=list(alg='pqnr'))

print(P1_3$M)

[1] "CP_PQNR (alternating Poisson regression using quasi-Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "-lno=1-i=1--jj:1----tmp_rho:381.795996,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.991961"
[1] "-lno=2-i=2--jj:1----tmp_rho:0.064525,lbfgsPos:2,nSizeLBFGS:3, kkt_violation:0.187459"
[1] "-lno=3-i=3--jj:1----tmp_rho:13.185215,lbfgsPos:3,nSizeLBFGS:3, kkt_violation:0.069235"
[1] "-lno=4-i=4--jj:1----tmp_rho:69.083059,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.051902"
[1] "-lno=5-i=5--jj:1----tmp_rho:262.168948,lbfgsPos:2,nSizeLBFGS:3, kkt_violation:0.008489"
[1] "-lno=6-i=6--jj:1----tmp_rho:11892.169051,lbfgsPos:3,nSizeLBFGS:3, kkt_violation:0.000717"
[1] "-lno=7-i=1--jj:2----tmp_rho:44.607092,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.870445"
[1] "-lno=8-i=2--jj:2----tmp_rho:0.001608,lbfgsPos:2,nSizeLBFGS:3, kkt_violation:37.633855"
[1] "-lno=9-i=3--jj:2----tmp_rho:0.003312,lbfgsPos:3,nSizeLBFGS:3, kkt_violation:0.735006"
[1] "-lno=10-i=4--jj:2----tmp_rho:274.045753,lbfgs

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = -2.925010e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 1.5062234e-05"
[1] " Total inner iterations = 489"
[1] " Total execution time = 0.75 secs"
$lambda
[1]  8.999982 47.999910

$u
$u[[1]]
          [,1]       [,2]
[1,] 0.1111111 0.00000000
[2,] 0.0000000 0.06250023
[3,] 0.0000000 0.93749977
[4,] 0.8888889 0.00000000

$u[[2]]
           [,1]       [,2]
 [1,] 0.0000000 0.04166672
 [2,] 0.0000000 0.04166672
 [3,] 0.0000000 0.02083337
 [4,] 0.0000000 0.02083337
 [5,] 0.0000000 0.04166672
 [6,] 0.0000000 0.02083337
 [7,] 0.0000000 0.02083337
 [8,] 0.0000000 0.04166672
 [9,] 0.0000000 0.02083337
[10,] 0.0000000 0.02083337
[11,] 0.0000000 0.02083337
[12,] 0.0000000 0.04166672
[13,] 0.2222261 0.00000000
[14,] 0.7777739 0.00000000
[15,] 0.0000000 0.04166672
[16,] 0.0000000 0.12499970
[17,] 0.0000000 0.02083337
[18,] 0.0000000 0.04166672
[19,] 0.0000000 0.04166672
[20,] 0.0000000 0.0416667

In [95]:
P1_4<- cp_apr(M,2,opts=list(alg='kappatol'))

print(P1_4$M)

            ## The alg is available in the doc but not yet supported as of current R -v

ERROR: Error in cp_apr(M, 2, opts = list(alg = "kappatol")): invalid alg:kappatol, possible values are mu,pdnr,pqnr


**CP_NMU implementation [ Fail ]**

In [66]:
set.seed(15)

P2 <- cp_nmu(M, R=2, opts=list(alg='pdnr'))

[1] 1 2 3
[1] "Nonnegative PARAFAC:"
[1] "n:1, calc spt_mttkrp..."
[1] "n:2, calc spt_mttkrp..."
[1] "n:3, calc spt_mttkrp..."


Warning message in sqrt(normX^2 + ktensor_norm(P)^2 - 2 * tt_innerprod(P, X)):
"NaNs produced"

[1] " Iter  1: fit = NaN fitdelta =     NaN, itime=0.00"
[1] "n:1, calc spt_mttkrp..."
[1] "n:2, calc spt_mttkrp..."
[1] "n:3, calc spt_mttkrp..."


Warning message in sqrt(normX^2 + ktensor_norm(P)^2 - 2 * tt_innerprod(P, X)):
"NaNs produced"

[1] " Iter  2: fit = NaN fitdelta =     NaN, itime=0.01"


ERROR: Error in if ((iter > 1) && (fitchange < fitchangetol)) {: missing value where TRUE/FALSE needed


In [73]:
tnsr=getTensor(dm)

[1] "The modes:"
[1]  4  4 25


**CP_01 implementation [ Fail ]**

In [87]:
df<- as.matrix(as.data.frame(cbind(Src=U17$SrcDevice, Dst=U17$DstDevice, Auth=U17$AuthType)))

head(df, 10)
class(df) 

Src,Dst,Auth
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp679250,Comp599155,NetworkLogon
Comp547871,ActiveDirectory,NetworkLogon
Comp679250,Comp599155,RemoteInteractive
Comp679250,Comp599155,NetworkLogon
Comp215429,Comp599155,NetworkLogon
Comp599155,Comp936659,NetworkLogon
Comp599155,Comp422203,NetworkLogon


[1] "matrix"

In [91]:
tnsr=getTensor(df)
subs=getTnsrijk(tnsr$X)
T<- list(subs=subs,vals=rep(1,nrow(subs)),size=c(length(unique(df[,1])), length(unique(df[,2])), length(unique(df[,3]))))
normT=sqrt(sum(T$vals))

set.seed(555)
P1<- cp_nmu(T, R=2)
res<- CP_R01(T, P1[[1]])

[1] "The modes:"
[1]  7  7 25
[1] 1 2 3
[1] "Nonnegative PARAFAC:"
[1] "n:1, calc spt_mttkrp..."
[1] "n:2, calc spt_mttkrp..."
[1] "n:3, calc spt_mttkrp..."


ERROR: Error in pracma::accumarray(newsubs, newvals, sz = newsiz): Argument 'sz' does not fit with 'subs'.


##### Same experiment on AuthType

In [10]:
df<- as.matrix(as.data.frame(cbind(Src=U17$SrcDevice, Dst=U17$DstDevice, Auth=U17$AuthType)))

t1<- tapply(df[,1], df[,1])
t2<- tapply(df[,2], df[,2])
t3<- tapply(df[,3], df[,3])

dt<- as.matrix(cbind(t1,t2,t3))

head(dt, 10)
head(df, 10)

t1,t2,t3
3,25,1
3,25,1
3,25,1
4,14,2
2,1,2
4,14,3
4,14,2
1,14,2
3,21,2
3,10,2


Src,Dst,Auth
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp679250,Comp599155,NetworkLogon
Comp547871,ActiveDirectory,NetworkLogon
Comp679250,Comp599155,RemoteInteractive
Comp679250,Comp599155,NetworkLogon
Comp215429,Comp599155,NetworkLogon
Comp599155,Comp936659,NetworkLogon
Comp599155,Comp422203,NetworkLogon


In [14]:
MT <- list(subs=dt, vals=rep(1, nrow(dt)), size=c(length(unique(dt[,1])), length(unique(dt[,2])), length(unique(dt[,3]))))

set.seed(25)
P<- cp_apr(MT,2,opts=list(alg='pdnr'))

print(P$M)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 64, KKT viol = 6.37e+00, obj = -5.31177865e+01, nz: 8, time:0.41"
[1] "   2. Ttl Inner Its: 87, KKT viol = 2.74e+00, obj = -3.18523726e+01, nz: 31, time:0.60"
[1] "   3. Ttl Inner Its: 36, KKT viol = 4.70e-02, obj = -3.18421374e+01, nz: 31, time:0.61"
[1] "   4. Ttl Inner Its: 63, KKT viol = 1.74e-02, obj = -3.18392438e+01, nz: 31, time:0.72"
[1] "   5. Ttl Inner Its: 34, KKT viol = 2.85e-04, obj = -3.18392436e+01, nz: 31, time:0.72"
[1] "   6. Ttl Inner Its: 33, KKT viol = 1.03e-04, obj = -3.18392436e+01, nz: 31, time:0.74"
[1] "   7. Ttl Inner Its: 32, KKT viol = 8.05e-05, obj = -3.18392436e+01, nz: 31, time:0.74"


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = -3.183924e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 8.0483328e-05"
[1] " Total inner iterations = 349"
[1] " Total execution time = 0.74 secs"
$lambda
[1]  8.999967 48.001164

$u
$u[[1]]
          [,1]      [,2]
[1,] 0.1111075 0.0000000
[2,] 0.0000000 0.0624941
[3,] 0.0000000 0.9375059
[4,] 0.8888925 0.0000000

$u[[2]]
           [,1]       [,2]
 [1,] 0.0000000 0.04166843
 [2,] 0.0000000 0.04166814
 [3,] 0.0000000 0.02083244
 [4,] 0.0000000 0.02083414
 [5,] 0.0000000 0.04166431
 [6,] 0.0000000 0.02083115
 [7,] 0.0000000 0.02083307
 [8,] 0.0000000 0.04166389
 [9,] 0.0000000 0.02083422
[10,] 0.0000000 0.02083280
[11,] 0.0000000 0.02083393
[12,] 0.0000000 0.04166565
[13,] 0.2222217 0.00000000
[14,] 0.7777783 0.00000000
[15,] 0.0000000 0.04166781
[16,] 0.0000000 0.12499697
[17,] 0.0000000 0.02083316
[18,] 0.0000000 0.04166484
[19,] 0.0000000 0.04166661
[20,] 0.0000000 0.04166838
[21

In [27]:
P2<- as.data.frame(P$M$u[2])
P2
dim(P2)

X1,X2
0.0000000,0.04166843
0.0000000,0.04166814
0.0000000,0.02083244
0.0000000,0.02083414
0.0000000,0.04166431
0.0000000,0.02083115
0.0000000,0.02083307
0.0000000,0.04166389
0.0000000,0.02083422
0.0000000,0.02083280


[1] 25  2

### User794682

In [67]:
U79 <- Auth[Auth$UserName=='User794682', ]
df<- as.data.frame(cbind(Src=U79$SrcDevice, Dst=U79$DstDevice, Fail=U79$Failure))

t1<- tapply(df$Src, df$Src)
t2<- tapply(df$Dst, df$Dst)
t3<- tapply(df$Fail, df$Fail)

dm<- as.data.frame(cbind(t1,t2,t3))
data<- as.data.frame(cbind(U79, samp=c(1:nrow(df))))
dm<- as.matrix(dm)

head(data, 10)
head(dm, 10)

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount,samp
628691,User794682,Comp700191,None,InteractiveLogon,0,1,1
650158,User794682,Comp700191,Comp550312,NetworkLogon,1,1,2
650813,User794682,Comp700191,Comp700191,TGS,0,1,3
731498,User794682,Comp700191,None,InteractiveLogon,1,1,4
759822,User794682,Comp679250,Comp679250,TGS,0,11,5
761120,User794682,Comp679250,ActiveDirectory,TGS,0,3,6
789856,User794682,Comp679250,None,InteractiveLogon,0,20,7
797499,User794682,Comp679250,None,CachedInteractive,1,10,8
830442,User794682,Comp679250,ActiveDirectory,NetworkLogon,0,1,9
1246255,User794682,Comp679250,Comp056180,TGS,0,1,10


t1,t2,t3
4,10,1
4,6,2
4,8,1
4,10,2
3,7,1
3,1,1
3,10,1
3,10,2
3,1,1
3,3,1


In [71]:
M <- list(subs=dm, vals=rep(1, nrow(dm)), size=c(length(unique(t1)), length(unique(t2)), length(unique(t3))))
invisible(M)

set.seed(5)
P1 <- cp_apr(M,R=2,opts=list(alg='pdnr'))

print(P1$M)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 32, KKT viol = 2.08e+01, obj = 1.70156747e+02, nz: 3, time:0.02"
[1] "   2. Ttl Inner Its: 41, KKT viol = 4.20e+00, obj = 2.31118805e+02, nz: 4, time:0.03"
[1] "   3. Ttl Inner Its: 36, KKT viol = 4.45e-01, obj = 2.37552586e+02, nz: 4, time:0.03"
[1] "   4. Ttl Inner Its: 41, KKT viol = 1.81e+00, obj = 2.38992092e+02, nz: 4, time:0.05"
[1] "   5. Ttl Inner Its: 24, KKT viol = 4.69e-01, obj = 2.39094981e+02, nz: 5, time:0.05"
[1] "   6. Ttl Inner Its: 28, KKT viol = 2.11e-02, obj = 2.39103658e+02, nz: 5, time:0.06"
[1] "   7. Ttl Inner Its: 32, KKT viol = 6.48e-03, obj = 2.39104749e+02, nz: 5, time:0.06"
[1] "   8. Ttl Inner Its: 32, KKT viol = 2.35e-03, obj = 2.39104936e+02, nz: 5, time:0.08"
[1] "   9. Ttl Inner Its: 31, KKT viol = 1.02e-03, obj = 2.39104969e+02, nz: 6, time:0.09"
[1] "  10. Ttl Inner Its: 31, KKT viol = 4.32e-04, obj 

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = 2.391050e+02 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 7.7620558e-05"
[1] " Total inner iterations = 366"
[1] " Total execution time = 0.09 secs"
$lambda
[1]  59.91216 105.08871

$u
$u[[1]]
           [,1]       [,2]
[1,] 0.05949301 0.03269286
[2,] 0.01669141 0.00000000
[3,] 0.68262340 0.79078322
[4,] 0.24119218 0.17652392

$u[[2]]
              [,1]        [,2]
 [1,] 1.835615e-02 0.160813603
 [2,] 0.000000e+00 0.038061346
 [3,] 0.000000e+00 0.038061346
 [4,] 1.669187e-02 0.000000000
 [5,] 0.000000e+00 0.038061346
 [6,] 1.836106e-01 0.000000000
 [7,] 0.000000e+00 0.266429412
 [8,] 5.407694e-10 0.095153367
 [9,] 0.000000e+00 0.009515337
[10,] 7.813414e-01 0.353904243

$u[[3]]
           [,1] [,2]
[1,] 0.09866591    1
[2,] 0.90133409    0




In [21]:
Pt1 <- as.data.frame(P1$M$u[1])
Pt2 <- as.data.frame(P1$M$u[2])

# s1<- sd(Pt1[,1]) + mean(Pt1[,1])
# s2<- sd(Pt2[,1]) + mean(Pt2[,1])

# print(c(s1, s2))

In [25]:
s<- IQR(Pt2[,1])
s1<- sd(Pt2[,1])
s
s1

[1] 0.01794008

[1] 0.2460404

In [59]:
R1<- as.numeric(row.names(Pt1[Pt1$X1>s1, ]))
R2<- as.numeric(row.names(Pt2[Pt2$X1>s2, ]))
R1
R2

[1] 3

[1] 10

In [60]:
dm<- as.data.frame(dm)
dmt <- dm[dm$t1%in%R1, ]
dmt <- dmt[dmt$t2%in%R2, ]
dmt <- dmt[dmt$t3==2, ]
dmt

,t1,t2,t3
8,3,10,2
11,3,10,2
14,3,10,2
18,3,10,2
21,3,10,2
27,3,10,2
31,3,10,2
33,3,10,2
39,3,10,2
45,3,10,2


In [61]:
nrow(dmt)/nrow(U79)*100

[1] 21.81818

In [67]:
N1<- as.numeric(row.names(dmt))
head(N1)
length(N1)

[1]  8 11 14 18 21 27

[1] 36

In [70]:
U79_anom <- data[data$samp%in%N1, ]
U79_anom <- U79_anom[, -ncol(U79_anom)]
head(U79_anom, 20)
identical(nrow(U79_anom), length(N1))

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
797499,User794682,Comp679250,None,CachedInteractive,1,10
1273517,User794682,Comp679250,None,InteractiveLogon,1,1
1385389,User794682,Comp679250,None,CachedInteractive,1,6
1604200,User794682,Comp679250,None,CachedInteractive,1,2
1721241,User794682,Comp679250,None,InteractiveLogon,1,1
1832590,User794682,Comp679250,None,CachedInteractive,1,16


[1] TRUE

In [61]:
DetectAnomalies <- function(x='User794682', s=1, q=0, m=1) {
    User <- Auth[Auth$UserName==x, ]
    n<- readline(prompt="Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F : ")
    if(n=='F') {
        df<- as.data.frame(cbind(Src=User$SrcDevice, Dst=User$DstDevice, Obj=User$Failure)) }
    if(n=='A') {
        df<- as.data.frame(cbind(Src=User$SrcDevice, Dst=User$DstDevice, Obj=User$AuthType)) }
    if(n!='A' & n!='F') {
        print("Unrecognized input.")
        cat("\n")
        stop()
    }
    
    t1<- tapply(df$Src, df$Src)
    t2<- tapply(df$Dst, df$Dst)
    t3<- tapply(df$Obj, df$Obj)

    dm<- as.data.frame(cbind(t1,t2,t3))
    data<- as.data.frame(cbind(User, samp=c(1:nrow(df))))
    dm<- as.matrix(dm)
    
    set.seed(5)
    M <- list(subs=dm, vals=rep(1, nrow(dm)), size=c(length(unique(t1)), length(unique(t2)), length(unique(t3))))
    invisible(M)
    P1 <- cp_apr(M,2,opts=list(alg='pdnr'))
    cat("\n")
    print(P1$M)
    cat("\n")
    
    Pt1 <- as.data.frame(P1$M$u[1])
    Pt2 <- as.data.frame(P1$M$u[2])
    Pt3 <- as.data.frame(P1$M$u[3])
    p1<- s*sd(Pt1[,1]) + m*mean(Pt1[,1]) + q*IQR(Pt1[,1])
    p2<- s*sd(Pt2[,1]) + m*mean(Pt2[,1]) + q*IQR(Pt2[,1])
    p3<- s*sd(Pt3[,1]) + m*mean(Pt3[,1]) + q*IQR(Pt3[,1])

    R1<- as.numeric(row.names(Pt1[Pt1$X1>p1, ]))
    R2<- as.numeric(row.names(Pt2[Pt2$X1>p2, ]))
    R3<- as.numeric(row.names(Pt3[Pt3$X1>p3, ]))
    
    
    dm<- as.data.frame(dm)
    dmt <- dm[dm$t1%in%R1, ]
    dmt <- dmt[dmt$t2%in%R2, ]
    dmt <- dmt[dmt$t3%in%R3, ]
    
    print(paste("The amount of anomalous data is", nrow(dmt)/nrow(User)*100, "% of the total data provided." ))
    cat("\n")
    
    N1<- as.numeric(row.names(dmt))
    User_anom <- data[data$samp%in%N1, ]
    User_anom <- User_anom[, -ncol(User_anom)]
    print(paste("Verdict for the integrity of the process is:", identical(nrow(User_anom), length(N1))))
    cat("\n\n")
    print("The resulting dataset is:")
    cat("\n")
    head(User_anom, round(nrow(User_anom)/10))
}

In [47]:
DetectAnomalies(s=0, m=0, q=1)

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F : F
[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 32, KKT viol = 2.08e+01, obj = 1.70156747e+02, nz: 3, time:0.03"
[1] "   2. Ttl Inner Its: 41, KKT viol = 4.20e+00, obj = 2.31118805e+02, nz: 4, time:0.07"
[1] "   3. Ttl Inner Its: 36, KKT viol = 4.45e-01, obj = 2.37552586e+02, nz: 4, time:0.10"
[1] "   4. Ttl Inner Its: 41, KKT viol = 1.81e+00, obj = 2.38992092e+02, nz: 4, time:0.13"
[1] "   5. Ttl Inner Its: 24, KKT viol = 4.69e-01, obj = 2.39094981e+02, nz: 5, time:0.13"
[1] "   6. Ttl Inner Its: 28, KKT viol = 2.11e-02, obj = 2.39103658e+02, nz: 5, time:0.16"
[1] "   7. Ttl Inner Its: 32, KKT viol = 6.48e-03, obj = 2.39104749e+02, nz: 5, time:0.17"
[1] "   8. Ttl Inner Its: 32, KKT viol = 2.35e-03, obj = 2.39104936e+02, nz: 5, time:0.21"
[1] "   9. Ttl Inner Its: 31, KKT viol = 1.02e-03, o

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = 2.391050e+02 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 7.7620558e-05"
[1] " Total inner iterations = 366"
[1] " Total execution time = 0.25 secs"


[1] "The amount of anomalous data is 22.4242424242424 % of the total data provided."

[1] "Verdict for the integrity of the process is: TRUE"


[1] "The resulting dataset is:"



,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
797499,User794682,Comp679250,None,CachedInteractive,1,10
1273517,User794682,Comp679250,None,InteractiveLogon,1,1
1385389,User794682,Comp679250,None,CachedInteractive,1,6
1604200,User794682,Comp679250,None,CachedInteractive,1,2


In [63]:
DetectAnomalies("AppService", s=1,m=0,q=0)

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F : F
[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 862, KKT viol = 8.79e+01, obj = 1.82825576e+04, nz: 263, time:2.06"
[1] "   2. Ttl Inner Its: 532, KKT viol = 1.60e+01, obj = 3.06047529e+04, nz: 272, time:2.33"
[1] "   3. Ttl Inner Its: 1078, KKT viol = 1.38e+00, obj = 3.54111463e+04, nz: 277, time:3.08"
[1] "   4. Ttl Inner Its: 1007, KKT viol = 1.28e+00, obj = 3.56548170e+04, nz: 269, time:4.19"
[1] "   5. Ttl Inner Its: 607, KKT viol = 1.40e+00, obj = 3.59742937e+04, nz: 268, time:5.61"
[1] "   6. Ttl Inner Its: 528, KKT viol = 2.48e+00, obj = 3.63073117e+04, nz: 272, time:6.03"
[1] "   7. Ttl Inner Its: 463, KKT viol = 3.84e-01, obj = 3.63317937e+04, nz: 274, time:6.46"
[1] "   8. Ttl Inner Its: 879, KKT viol = 1.24e+00, obj = 3.63412616e+04, nz: 276, time:7.91"
[1] "   9. Ttl Inner Its: 

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = 3.634715e+04 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 9.9887570e-05"
[1] " Total inner iterations = 10358"
[1] " Total execution time = 9.91 secs"

$lambda
[1] 11570.124  4221.026

$u
$u[[1]]
               [,1]         [,2]
  [1,] 6.050168e-04 0.000000e+00
  [2,] 5.185799e-04 1.664249e-08
  [3,] 6.050169e-04 0.000000e+00
  [4,] 5.185858e-04 0.000000e+00
  [5,] 5.185806e-04 1.457767e-08
  [6,] 1.037172e-03 0.000000e+00
  [7,] 6.050169e-04 0.000000e+00
  [8,] 1.296465e-03 0.000000e+00
  [9,] 6.914479e-04 0.000000e+00
 [10,] 4.321549e-04 0.000000e+00
 [11,] 8.643097e-05 0.000000e+00
 [12,] 9.507407e-04 0.000000e+00
 [13,] 5.185859e-04 0.000000e+00
 [14,] 8.643098e-05 0.000000e+00
 [15,] 2.592930e-04 0.000000e+00
 [16,] 4.321484e-04 1.781658e-08
 [17,] 1.728619e-04 0.000000e+00
 [18,] 2.592930e-04 0.000000e+00
 [19,] 1.123603e-03 0.000000e+00
 [20,] 3.509104e-02 0.000000e+00
 [21,] 2.

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
2602,AppService,Comp154659,None,Batch,0,2
2831,AppService,Comp093748,ActiveDirectory,TGS,0,4
7593,AppService,Comp397526,ActiveDirectory,TGT,0,31
8804,AppService,Comp367857,None,Batch,0,1
13046,AppService,Comp859239,ActiveDirectory,TGT,0,29
15138,AppService,Comp542689,None,Service,0,2
17810,AppService,Comp484824,ActiveDirectory,TGT,0,15
18521,AppService,Comp581624,ActiveDirectory,TGT,0,22
19448,AppService,Comp484824,None,Service,0,1
24155,AppService,Comp843020,ActiveDirectory,NetworkLogon,0,2


To estimate FPR:
Both for my CP_APR and for AE (assuming it too was uniformly applied on the whole 90 days) we look at how many anomalies the models detect Days 1 - 56, since we know there aren't supposed to be any in that time period. 


##### Testing the interaction with non-malicious usernames:  (Failure based approach)

1. **Comp599155$** [Success]

In [56]:
DetectAnomalies("Comp599155$", m=0, s=0, q=1)

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F : F
[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 364, KKT viol = 8.09e+01, obj = 1.07618959e+03, nz: 11, time:0.20"
[1] "   2. Ttl Inner Its: 186, KKT viol = 2.00e+00, obj = 1.10856301e+03, nz: 12, time:0.22"
[1] "   3. Ttl Inner Its: 990, KKT viol = 1.01e+01, obj = 1.13771694e+03, nz: 179, time:0.65"
[1] "   4. Ttl Inner Its: 182, KKT viol = 2.18e-02, obj = 1.13771694e+03, nz: 179, time:0.68"
[1] "   5. Ttl Inner Its: 367, KKT viol = 2.00e-02, obj = 1.13774294e+03, nz: 181, time:0.76"
[1] "   6. Ttl Inner Its: 183, KKT viol = 2.14e-04, obj = 1.13774294e+03, nz: 181, time:0.84"
[1] "   7. Ttl Inner Its: 195, KKT viol = 1.97e-04, obj = 1.13774294e+03, nz: 181, time:0.87"
[1] "   8. Ttl Inner Its: 182, KKT viol = 9.45e-05, obj = 1.13774294e+03, nz: 181, time:0.89"


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = 1.137743e+03 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 9.4509667e-05"
[1] " Total inner iterations = 2649"
[1] " Total execution time = 0.89 secs"




Warning message in DetectAnomalies("Comp599155$", m = 0, s = 0, q = 1):
"NAs introduced by coercion"

[1] "The amount of anomalous data is 0 % of the total data provided."

[1] "Verdict for the integrity of the process is: TRUE"


[1] "The resulting dataset is:"



UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount


2. **User035855** [Success]	

In [62]:
DetectAnomalies("User035855", m=1, s=0, q=1)

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F : A
[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 72, KKT viol = 1.76e+01, obj = 1.76270867e+03, nz: 22, time:0.06"
[1] "   2. Ttl Inner Its: 105, KKT viol = 1.03e+03, obj = 2.19499882e+03, nz: 26, time:0.12"
[1] "   3. Ttl Inner Its: 36, KKT viol = 1.00e+00, obj = 2.19499882e+03, nz: 26, time:0.14"
[1] "   4. Ttl Inner Its: 110, KKT viol = 1.00e+00, obj = 2.34159489e+03, nz: 27, time:0.20"
[1] "   5. Ttl Inner Its: 78, KKT viol = 4.54e-01, obj = 2.34795346e+03, nz: 28, time:0.25"
[1] "   6. Ttl Inner Its: 44, KKT viol = 4.08e-02, obj = 2.34800552e+03, nz: 28, time:0.27"
[1] "   7. Ttl Inner Its: 72, KKT viol = 5.15e-03, obj = 2.34801054e+03, nz: 28, time:0.30"
[1] "   8. Ttl Inner Its: 41, KKT viol = 4.99e-04, obj = 2.34801085e+03, nz: 28, time:0.31"
[1] "   9. Ttl Inner Its: 36, KKT viol = 9

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = 2.348011e+03 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 9.4546249e-05"
[1] " Total inner iterations = 594"
[1] " Total execution time = 0.33 secs"

$lambda
[1] 295.6369 796.3716

$u
$u[[1]]
           [,1]        [,2]
[1,] 0.00000000 0.032650226
[2,] 0.02723483 0.015003319
[3,] 0.00000000 0.001255777
[4,] 0.87805939 0.947323347
[5,] 0.00000000 0.003767331
[6,] 0.09470578 0.000000000

$u[[2]]
              [,1]        [,2]
 [1,] 2.727952e-01 0.146102236
 [2,] 0.000000e+00 0.138130623
 [3,] 0.000000e+00 0.003767197
 [4,] 0.000000e+00 0.113015953
 [5,] 0.000000e+00 0.070321025
 [6,] 0.000000e+00 0.046462104
 [7,] 0.000000e+00 0.092924221
 [8,] 0.000000e+00 0.001255732
 [9,] 0.000000e+00 0.030137577
[10,] 0.000000e+00 0.001255732
[11,] 0.000000e+00 0.060275162
[12,] 8.943705e-08 0.006278443
[13,] 0.000000e+00 0.001255732
[14,] 0.000000e+00 0.007534392
[15,] 0.000000e+00 0.071576751
[16,]

UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount


3. **User762066** [Success]

In [40]:
DetectAnomalies("User762066")

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F : F
[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 66, KKT viol = 3.27e+01, obj = 4.01597315e+03, nz: 20, time:0.03"
[1] "   2. Ttl Inner Its: 88, KKT viol = 2.44e+00, obj = 4.97180766e+03, nz: 19, time:0.06"
[1] "   3. Ttl Inner Its: 81, KKT viol = 4.13e-01, obj = 5.04227557e+03, nz: 25, time:0.08"
[1] "   4. Ttl Inner Its: 70, KKT viol = 3.10e-01, obj = 5.04677665e+03, nz: 26, time:0.11"
[1] "   5. Ttl Inner Its: 60, KKT viol = 2.17e-02, obj = 5.04690132e+03, nz: 26, time:0.13"
[1] "   6. Ttl Inner Its: 67, KKT viol = 5.71e-03, obj = 5.04691040e+03, nz: 26, time:0.14"
[1] "   7. Ttl Inner Its: 63, KKT viol = 1.14e-03, obj = 5.04691078e+03, nz: 26, time:0.16"
[1] "   8. Ttl Inner Its: 39, KKT viol = 2.28e-04, obj = 5.04691081e+03, nz: 26, time:0.16"
[1] "   9. Ttl Inner Its: 33, KKT viol = 9.2

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = 5.046911e+03 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 9.2299302e-05"
[1] " Total inner iterations = 567"
[1] " Total execution time = 0.16 secs"


[1] "The amount of anomalous data is 0 % of the total data provided."

[1] "Verdict for the integrity of the process is: TRUE"


[1] "The resulting dataset is:"



UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount


4. **Scanner** [Success]

In [43]:
DetectAnomalies("Scanner")

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F : F
[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 20150, KKT viol = 9.57e+01, obj = -6.44404363e+03, nz: 8606, time:36.21"
[1] "   2. Ttl Inner Its: 11123, KKT viol = 8.51e+01, obj = 4.26322557e+04, nz: 8312, time:38.39"
[1] "   3. Ttl Inner Its: 10422, KKT viol = 2.80e+00, obj = 4.84416781e+04, nz: 8213, time:43.96"
[1] "   4. Ttl Inner Its: 26728, KKT viol = 2.23e+00, obj = 6.81036503e+04, nz: 9058, time:57.46"
[1] "   5. Ttl Inner Its: 12506, KKT viol = 8.80e-01, obj = 6.90202380e+04, nz: 8991, time:61.21"
[1] "   6. Ttl Inner Its: 16006, KKT viol = 4.13e-01, obj = 6.93166752e+04, nz: 9127, time:65.27"
[1] "   7. Ttl Inner Its: 14749, KKT viol = 3.52e-01, obj = 6.95809042e+04, nz: 9163, time:68.52"
[1] "   8. Ttl Inner Its: 17791, KKT viol = 3.78e-01, obj = 6.98211693e+04, nz: 9135, time:72

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = 7.019881e+04 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 9.9741544e-05"
[1] " Total inner iterations = 251048"
[1] " Total execution time = 96.19 secs"


[1] "The amount of anomalous data is 0 % of the total data provided."

[1] "Verdict for the integrity of the process is: TRUE"


[1] "The resulting dataset is:"



UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount


5. **User631552** [Success]

In [44]:
DetectAnomalies("User631552")

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F : F
[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 50, KKT viol = 1.50e+02, obj = 2.41316104e+03, nz: 13, time:0.06"
[1] "   2. Ttl Inner Its: 35, KKT viol = 9.18e+00, obj = 3.71513711e+03, nz: 13, time:0.06"
[1] "   3. Ttl Inner Its: 79, KKT viol = 1.00e+00, obj = 3.90289901e+03, nz: 15, time:0.09"
[1] "   4. Ttl Inner Its: 74, KKT viol = 9.03e-01, obj = 3.92702536e+03, nz: 18, time:0.11"
[1] "   5. Ttl Inner Its: 57, KKT viol = 1.00e+00, obj = 3.93050488e+03, nz: 18, time:0.12"
[1] "   6. Ttl Inner Its: 47, KKT viol = 1.81e+00, obj = 3.93581881e+03, nz: 19, time:0.14"
[1] "   7. Ttl Inner Its: 28, KKT viol = 8.96e-01, obj = 3.93683683e+03, nz: 20, time:0.14"
[1] "   8. Ttl Inner Its: 26, KKT viol = 1.15e-02, obj = 3.93683912e+03, nz: 20, time:0.15"
[1] "   9. Ttl Inner Its: 58, KKT viol = 8.0

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = 3.936853e+03 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 9.1891342e-05"
[1] " Total inner iterations = 577"
[1] " Total execution time = 0.19 secs"


[1] "The amount of anomalous data is 0 % of the total data provided."

[1] "Verdict for the integrity of the process is: TRUE"


[1] "The resulting dataset is:"



UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount


In [64]:
length(unique(Auth$UserName))

[1] 28815